## 将csv合并为一个parquet文件

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
将 /d:/workspace/xiaoyao/redis/ 目录下的所有 stock_***.csv 文件合并为一个 parquet 文件
确保与现有 /d:/workspace/xiaoyao/data/stock_daily_price.parquet 保持字段、压缩方式一致
"""

import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
import glob


def merge_stock_csv_to_parquet(csv_dir,stock_code, output_parquet_file):
    """
    合并指定目录下的所有 stock_***.csv 文件到单个 parquet 文件
    
    Args:
        csv_dir: CSV 文件所在目录
        output_parquet_file: 输出的 parquet 文件路径
    """
    print(f"📁 开始处理目录: {csv_dir}")
    
    # 获取所有 stock_***.csv 文件
    csv_pattern = os.path.join(csv_dir, f"stock_minutely_price_{stock_code}*.csv")
    csv_files = glob.glob(csv_pattern)
    
    if not csv_files:
        print(f"❌ 未找到 stock_minutely_price_{stock_code}*.csv 文件")
        return False
    
    print(f"📊 找到 {len(csv_files)} 个 CSV 文件")
    
    # 按文件名排序（确保按日期顺序处理）
    csv_files.sort()
    
    # 读取并合并所有 CSV 文件
    all_dataframes = []
    total_records = 0
    
    for i, csv_file in enumerate(csv_files, 1):
        filename = os.path.basename(csv_file)
        print(f"正在处理 ({i}/{len(csv_files)}): {filename}")
        
        try:
            # 读取 CSV 文件
            df = pd.read_csv(csv_file)
            
            # 数据验证和清洗
            # 确保 date 列是 datetime 类型
            df['date'] = pd.to_datetime(df['date'])
            
            # 确保数值列的数据类型正确
            numeric_columns = ['open', 'close', 'low', 'high', 'volume']
            
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # 删除无效数据
            df = df.dropna(subset=['date', 'stock_code','time'])
            
            all_dataframes.append(df)
            total_records += len(df)
            print(f"  ✅ 成功读取 {len(df)} 条记录")
            
        except Exception as e:
            print(f"  ❌ 处理失败: {e}")
            continue
    
    if not all_dataframes:
        print("❌ 没有成功读取任何数据")
        return False
    
    print(f"\n📊 合并所有数据...")
    # 合并所有数据框
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    # 去重（按 date + stock_code + time）
    combined_df = combined_df.drop_duplicates(subset=['date', 'stock_code','time'])
    
    # 按日期和股票代码排序
    combined_df = combined_df.sort_values(['date', 'stock_code','time']).reset_index(drop=True)
    
    print(f"📈 总计 {len(combined_df)} 条记录（去重后）")
    
    # 确保输出目录存在
    output_dir = os.path.dirname(output_parquet_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 创建输出目录: {output_dir}")
    
    # 转换为 pyarrow Table
    table = pa.Table.from_pandas(combined_df)
    
    # 使用与目标文件相同的压缩方式 (snappy) 和格式写入 parquet
    try:
        pq.write_table(
            table, 
            output_parquet_file,
            compression='snappy',
            version='2.6',  # 使用较新的 parquet 版本
            use_dictionary=True,
            write_batch_size=64 * 1024 * 1024  # 64MB batch size for better performance
        )
        
        print(f"✅ 成功保存到: {output_parquet_file}")
        print(f"📊 文件大小: {os.path.getsize(output_parquet_file) / (1024*1024):.2f} MB")
        
        return True
        
    except Exception as e:
        print(f"❌ 保存 parquet 文件失败: {e}")
        return False

## 将新生成的 merged_stock_data.parquet 与现有的 stock_daily_price.parquet 合并

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
将新生成的 merged_stock_data.parquet 与现有的 stock_daily_price.parquet 合并
"""

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import shutil

def merge_parquet_files(stock_code):
    """
    合并两个 parquet 文件
    
    Args:
        existing_file: 现有的 parquet 文件路径
        new_file: 新的 parquet 文件路径  
        output_file: 输出的合并文件路径
    """

    existing_file = f"d:/workspace/xiaoyao/data/stock_minutely_price/{stock_code}.parquet"
    new_file = f"d:/workspace/xiaoyao/redis/minutely/{stock_code}_merged.parquet"
    output_file = f"d:/workspace/xiaoyao/data/stock_minutely_price/{stock_code}.parquet"


    # 检查文件是否存在
    if not os.path.exists(existing_file):
        print(f"❌ 现有文件不存在: {existing_file}")
        #直接用先用文件去覆盖
        shutil.move(new_file, existing_file)
        return
    
    if not os.path.exists(new_file):
        print(f"❌ 新文件不存在: {new_file}")
        return

    print("📊 开始合并 parquet 文件...")
    
    try:
        # 读取现有数据
        print(f"📖 读取现有文件: {existing_file}")
        existing_df = pd.read_parquet(existing_file)
        print(f"   现有数据行数: {len(existing_df)}")
        
        # 读取新数据
        print(f"📖 读取新文件: {new_file}")
        new_df = pd.read_parquet(new_file)
        print(f"   新数据行数: {len(new_df)}")
        
        # 合并数据
        print("🔄 合并数据中...")
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        
        # 去重（按 date + stock_code）
        print("🧹 去重处理...")
        combined_df = combined_df.drop_duplicates(subset=['date', 'stock_code', 'time'])
        
        # 排序
        print("📅 按日期排序...")
        combined_df = combined_df.sort_values(['date', 'stock_code', 'time']).reset_index(drop=True)
        
        print(f"📈 合并后总行数: {len(combined_df)}")
        
        # 确保输出目录存在
        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 转换为 pyarrow Table
        table = pa.Table.from_pandas(combined_df)
        
        # 写入 parquet（使用与源文件相同的格式）
        print(f"💾 保存合并结果: {output_file}")
        pq.write_table(
            table,
            output_file,
            compression='snappy',
            version='2.6',
            use_dictionary=True,
            write_batch_size=64 * 1024 * 1024
        )
        
        print(f"✅ 合并完成！文件大小: {os.path.getsize(output_file) / (1024*1024):.2f} MB")
        
        # 验证结果
        print("\n📋 验证结果:")
        result_df = pd.read_parquet(output_file)
        print(f"   最终行数: {len(result_df)}")
        print(f"   日期范围: {result_df['date'].min()} 到 {result_df['date'].max()}")
        print(f"   股票数量: {result_df['stock_code'].nunique()}")
        

        # 移动文件，清理
        # # 检查源文件是否存在
        # source_file_path = f'./minutely/{stock_code}.parquet'
        # target_file_path = f'D:/workspace/xiaoyao/redis/data/minutely/{stock_code}.parquet'
        # if os.path.exists(source_file_path):
        #     # 移动文件
        #     shutil.move(source_file_path, target_file_path)
        #     print(f"Moved: {file_to_move} to {target_dir}")
        # else:
        #     print(f"File not found: {source_file_path}")

        return True
        
    except Exception as e:
        print(f"❌ 合并失败: {e}")
        return False

## 删除已使用的csv

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
删除指定目录下的 stock_***.csv 文件
"""

import os
import glob
from pathlib import Path


def delete_stock_csv_files(stock_code,target_directory=r'D:\workspace\xiaoyao\redis\minutely'):
    #删除满足模式的所有文件
    pattern = f"stock_minutely_price_{stock_code}*.csv"
    files = glob.glob(os.path.join(target_directory, pattern))
    for file in files:
        try:
            os.remove(file)
            print(f"已删除：{file}")
        except Exception as e:
            print(f"删除 {file} 失败：{e}")

## 下载数据

In [4]:
import redis
import pickle
import time
import uuid
import pandas as pd
from io import StringIO
from typing import Any, Optional
from datetime import datetime, timedelta

class RemoteSender:
    def __init__(self, host='*', port=6379, password='*'):
        self.redis = redis.Redis(
            host=host, port=port, password=password,
            decode_responses=False
        )
        self.task_queue = 'function_calls'
        self.result_queue = 'function_results'
        self._test_connection()
        print(f"✅ 发送端pandas版本：{pd.__version__}")

    def _test_connection(self):
        try:
            self.redis.ping()
            print("✅ 发送端：Redis连接成功")
        except Exception as e:
            print(f"❌ 发送端：连接失败 - {e}")
            raise

    def call_remote_function(self, func_name: str, *args, **kwargs) -> Any:
        task_id = f"task_{uuid.uuid4().hex[:8]}"
        task = {
            'func_name': func_name,
            'args': args,
            'kwargs': kwargs,
            'task_id': task_id
        }
        self.redis.rpush(self.task_queue, pickle.dumps(task))
        print(f"📤 已调用远程函数：{func_name}（任务ID：{task_id}）")
        return self._get_result(task_id)

    def _get_result(self, task_id: str, timeout=300) -> Any:
        start_time = time.time()
        while time.time() - start_time < timeout:
            result_data = self.redis.blpop(self.result_queue, timeout=10)
            if not result_data:
                continue

            _, res_bytes = result_data
            result = pickle.loads(res_bytes)
            if result['task_id'] == task_id:
                if result['status'] == 'success':
                    return result['result']  # 返回CSV字符串
                else:
                    raise Exception(f"远程执行失败：{result['error']}")
            self.redis.rpush(self.result_queue, res_bytes)
        raise TimeoutError("任务超时")

    def save_to_csv(self, csv_str: Optional[str], filename: str) -> bool:
        """将CSV字符串保存为本地CSV文件（替代Parquet）"""
        if not csv_str:
            print("⚠️ 数据为空，不保存")
            return False
        try:
            # 从CSV字符串恢复DataFrame（兼容所有pandas版本）
            df = pd.read_csv(StringIO(csv_str))
            # 保存为CSV文件
            df.to_csv(filename, index=False)
            print(f"✅ 保存成功：{filename}（{len(df)}条记录）")
            return True
        except Exception as e:
            print(f"❌ 保存失败：{e}")
            return False

def generate_date_range(start_date_str: str, end_date_str: str) -> list:
    """生成从开始日期到结束日期的所有日期字符串（YYYYMMDD格式）"""
    dates = []
    try:
        start_date = datetime.strptime(start_date_str, '%Y%m%d')
        end_date = datetime.strptime(end_date_str, '%Y%m%d')
        
        if start_date > end_date:
            raise ValueError("开始日期晚于结束日期")
            
        current_date = start_date
        while current_date <= end_date:
            dates.append(current_date.strftime('%Y%m%d'))
            current_date += timedelta(days=1)
    except Exception as e:
        print(f"日期处理错误：{e}")
    return dates

## 发布任务

In [10]:
import os
import uuid
import redis
import json
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

# 工具函数：生成日期范围
def generate_date_range(start_date_str: str, end_date_str: str) -> list:
    dates = []
    try:
        start_date = datetime.strptime(start_date_str, '%Y%m%d')
        end_date = datetime.strptime(end_date_str, '%Y%m%d')
        if start_date > end_date:
            raise ValueError("开始日期晚于结束日期")
        current_date = start_date
        while current_date <= end_date:
            dates.append(current_date.strftime('%Y%m%d'))
            current_date += timedelta(days=1)
    except Exception as e:
        print(f"日期处理错误：{e}")
    return dates

# 任务发布管理器
class TaskPublisher:
    def __init__(self, host, port, password):
        self.redis = redis.Redis(
            host=host, port=port, password=password,
            decode_responses=False
        )
        self.task_queue = 'function_calls'
        self.task_map_key = 'minute_task_map'  # Redis中存储任务映射的Hash键
        self._test_connection()

    def _test_connection(self):
        try:
            self.redis.ping()
            print("✅ Redis连接成功")
        except Exception as e:
            print(f"❌ Redis连接失败：{e}")
            raise SystemExit(1)

    def publish_tasks(self, stock_code, date_list):
        """为单只股票发布所有日期的任务"""
        task_ids = []
        for date in date_list:
            task_id = f"task_{uuid.uuid4().hex[:8]}"
            # 保存任务映射到Redis Hash
            task_info = {
                'stock_code': stock_code,
                'date': date,
                'status': 'pending'
            }
            self.redis.hset(
                self.task_map_key,
                task_id.encode('utf-8'),
                json.dumps(task_info).encode('utf-8')
            )
            # 发布任务到队列
            task = {
                'func_name': 'fetch_minute_stock_data',
                'args': (date, [stock_code]),
                'kwargs': {},
                'task_id': task_id
            }
            self.redis.rpush(self.task_queue, pickle.dumps(task))
            task_ids.append(task_id)
        return len(task_ids)

if __name__ == "__main__":
    # 1. 读取Redis配置
    with open('redis.conf', 'r') as f:
        host = 'localhost'
        port = 6379
        password = ''
        for line in f:
            if line.startswith('host='):
                host = line.split('=')[1].strip()
            elif line.startswith('port='):
                port = int(line.split('=')[1].strip())
            elif line.startswith('password='):
                password = line.split('=')[1].strip()

    # 2. 初始化发布器
    publisher = TaskPublisher(host, port, password)

    # 3. 配置取数范围
    daily_df = pd.read_parquet('../data/stock_daily_price.parquet')
    stock_code_list = daily_df['stock_code'].unique()  # 所有股票
    start_date = '20250901'  # 开始日期
    end_date = '20250919' #(datetime.today() - timedelta(days=1)).strftime('%Y%m%d')  # 结束日期（昨天）
    date_list = generate_date_range(start_date, end_date)

    # 4. 批量发布任务
    print(f"=== 开始发布任务：{len(stock_code_list)}只股票 × {len(date_list)}天 ===")
    total_published = 0
    for stock_code in tqdm(stock_code_list, desc="发布进度"):
        count = publisher.publish_tasks(stock_code, date_list)
        total_published += count

    print(f"=== 发布完成，共{total_published}个任务 ===")
    print(f"任务映射已保存到Redis（键：minute_task_map）")

✅ Redis连接成功
=== 开始发布任务：5447只股票 × 19天 ===


发布进度: 100%|██████████| 5447/5447 [17:31<00:00,  5.18it/s]

=== 发布完成，共103493个任务 ===
任务映射已保存到Redis（键：minute_task_map）


In [11]:
import os
import time
import redis
import json
import pickle
import pandas as pd
from io import StringIO
from tqdm import tqdm

# 结果拉取管理器
class ResultFetcher:
    def __init__(self, host, port, password):
        self.redis = redis.Redis(
            host=host, port=port, password=password,
            decode_responses=False
        )
        self.result_queue = 'function_results'
        self.task_map_key = 'minute_task_map'  # 与发布模块一致的Hash键
        self.save_dir = './minutely'  # 结果保存目录
        os.makedirs(self.save_dir, exist_ok=True)
        self._test_connection()

    def _test_connection(self):
        try:
            self.redis.ping()
            print("✅ Redis连接成功")
        except Exception as e:
            print(f"❌ Redis连接失败：{e}")
            raise SystemExit(1)

    def get_pending_tasks(self):
        """获取所有未完成的任务"""
        raw_data = self.redis.hgetall(self.task_map_key)
        tasks = {}
        for task_id_bytes, info_bytes in raw_data.items():
            task_id = task_id_bytes.decode('utf-8')
            task_info = json.loads(info_bytes.decode('utf-8'))
            if task_info['status'] == 'pending':
                tasks[task_id] = task_info
        return tasks

    def save_to_csv(self, csv_str, stock_code, date):
        """保存结果到CSV文件"""
        if not csv_str:
            return False
        filename = f"{self.save_dir}/stock_minutely_price_{stock_code}_{date}.csv"
        try:
            df = pd.read_csv(StringIO(csv_str))
            df.to_csv(filename, index=False)
            return True
        except Exception as e:
            print(f"❌ 保存 {filename} 失败：{e}")
            return False

    def fetch_results(self, timeout_per_task=300):
        """拉取并处理结果"""
        pending_tasks = self.get_pending_tasks()
        total_tasks = len(pending_tasks)
        if total_tasks == 0:
            print("ℹ️ 没有待处理的任务")
            return

        print(f"=== 开始拉取结果：共{total_tasks}个任务 ===")
        completed = 0
        start_time = time.time()
        max_timeout = timeout_per_task * total_tasks

        with tqdm(total=total_tasks, desc="拉取进度") as pbar:
            while completed < total_tasks and (time.time() - start_time) < max_timeout:
                # 从结果队列获取数据
                result_data = self.redis.blpop(self.result_queue, timeout=10)
                if not result_data:
                    continue

                _, res_bytes = result_data
                result = pickle.loads(res_bytes)
                task_id = result['task_id']

                # 检查任务是否在待处理列表中
                if task_id not in pending_tasks:
                    continue

                # 处理结果
                task_info = pending_tasks[task_id]
                stock_code = task_info['stock_code']
                date = task_info['date']

                if result['status'] == 'success' and result['result']:
                    self.save_to_csv(result['result'], stock_code, date)
                    completed += 1
                    pbar.update(1)
                    # 从Redis删除已完成任务
                    self.redis.hdel(self.task_map_key, task_id.encode('utf-8'))
                else:
                    # 更新失败状态
                    task_info['status'] = 'failed'
                    task_info['error'] = result.get('error', '未知错误')
                    self.redis.hset(
                        self.task_map_key,
                        task_id.encode('utf-8'),
                        json.dumps(task_info).encode('utf-8')
                    )

        print(f"=== 拉取结束：成功{completed}/{total_tasks}个任务 ===")

if __name__ == "__main__":
    # 1. 读取Redis配置
    with open('redis.conf', 'r') as f:
        host = 'localhost'
        port = 6379
        password = ''
        for line in f:
            if line.startswith('host='):
                host = line.split('=')[1].strip()
            elif line.startswith('port='):
                port = int(line.split('=')[1].strip())
            elif line.startswith('password='):
                password = line.split('=')[1].strip()

    # 2. 拉取结果
    fetcher = ResultFetcher(host, port, password)
    fetcher.fetch_results(timeout_per_task=300)  # 每个任务超时300秒

✅ Redis连接成功


=== 开始拉取结果：共121857个任务 ===


拉取进度:  13%|█▎        | 15985/121857 [13:18<1:28:06, 20.03it/s]


KeyboardInterrupt: 

In [ ]:
import os
import glob
import shutil
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

if __name__ == "__main__":
    # 1. 获取需要合并的股票列表（从CSV文件中提取）
    csv_dir = './minutely'
    csv_files = glob.glob(os.path.join(csv_dir, 'stock_minutely_price_*.csv'))
    if not csv_files:
        print("❌ 未找到任何CSV文件，无需合并")
        raise SystemExit(1)

    # 提取唯一的股票代码
    stock_codes = set()
    for file in csv_files:
        # 从文件名提取股票代码（格式：stock_minutely_price_{code}_{date}.csv）
        parts = os.path.basename(file).split('_')
        if len(parts) >= 4:
            stock_codes.add(parts[3])
    stock_codes = list(stock_codes)

    # 2. 按股票合并数据
    print(f"=== 开始合并数据：共{len(stock_codes)}只股票 ===")
    for stock_code in tqdm(stock_codes, desc="合并进度"):
        # 2.1 CSV合并为Parquet
        merge_success = merge_stock_csv_to_parquet(
            csv_dir=csv_dir,
            stock_code=stock_code,
            output_parquet_file=f'./minutely/{stock_code}_merged.parquet'
        )
        if not merge_success:
            continue

        # 2.2 合并到现有数据
        merge_parquet_files(stock_code)

        # 2.3 清理临时文件
        os.remove(f'./minutely/{stock_code}_merged.parquet')
        delete_stock_csv_files(stock_code)\

    print("=== 所有数据合并完成 ===")

=== 开始合并数据：共3937只股票 ===


合并进度:   0%|          | 0/3937 [00:00<?, ?it/s]

📁 开始处理目录: ./minutely
📊 找到 1 个 CSV 文件
正在处理 (1/1): stock_minutely_price_002623.XSHE_20250919.csv
  ✅ 成功读取 240 条记录

📊 合并所有数据...
📈 总计 240 条记录（去重后）
✅ 成功保存到: ./minutely/002623.XSHE_merged.parquet
📊 文件大小: 0.01 MB
📊 开始合并 parquet 文件...
📖 读取现有文件: d:/workspace/xiaoyao/data/stock_minutely_price/002623.XSHE.parquet


合并进度:   0%|          | 1/3937 [00:00<58:45,  1.12it/s]

   现有数据行数: 2160
📖 读取新文件: d:/workspace/xiaoyao/redis/minutely/002623.XSHE_merged.parquet
   新数据行数: 240
🔄 合并数据中...
🧹 去重处理...
📅 按日期排序...
📈 合并后总行数: 2400
💾 保存合并结果: d:/workspace/xiaoyao/data/stock_minutely_price/002623.XSHE.parquet
✅ 合并完成！文件大小: 0.04 MB

📋 验证结果:
   最终行数: 2400
   日期范围: 2025-09-19 00:00:00 到 2025-10-10 00:00:00
   股票数量: 1
已删除：D:\workspace\xiaoyao\redis\minutely\stock_minutely_price_002623.XSHE_20250919.csv
📁 开始处理目录: ./minutely
📊 找到 1 个 CSV 文件
正在处理 (1/1): stock_minutely_price_300625.XSHE_20250919.csv
  ✅ 成功读取 240 条记录

📊 合并所有数据...
📈 总计 240 条记录（去重后）
✅ 成功保存到: ./minutely/300625.XSHE_merged.parquet
📊 文件大小: 0.01 MB
📊 开始合并 parquet 文件...
📖 读取现有文件: d:/workspace/xiaoyao/data/stock_minutely_price/300625.XSHE.parquet


...

合并进度:   2%|▏         | 76/3937 [00:39<56:19,  1.14it/s]

   现有数据行数: 2160
📖 读取新文件: d:/workspace/xiaoyao/redis/minutely/300985.XSHE_merged.parquet
   新数据行数: 240
🔄 合并数据中...
🧹 去重处理...
📅 按日期排序...
📈 合并后总行数: 2400
💾 保存合并结果: d:/workspace/xiaoyao/data/stock_minutely_price/300985.XSHE.parquet
✅ 合并完成！文件大小: 0.04 MB

📋 验证结果:
   最终行数: 2400
   日期范围: 2025-09-19 00:00:00 到 2025-10-10 00:00:00
   股票数量: 1
已删除：D:\workspace\xiaoyao\redis\minutely\stock_minutely_price_300985.XSHE_20250919.csv
📁 开始处理目录: ./minutely
📊 找到 1 个 CSV 文件
正在处理 (1/1): stock_minutely_price_688275.XSHG_20250919.csv
  ✅ 成功读取 240 条记录

📊 合并所有数据...
📈 总计 240 条记录（去重后）
✅ 成功保存到: ./minutely/688275.XSHG_merged.parquet
📊 文件大小: 0.01 MB
📊 开始合并 parquet 文件...
📖 读取现有文件: d:/workspace/xiaoyao/data/stock_minutely_price/688275.XSHG.parquet


合并进度:   2%|▏         | 77/3937 [00:39<54:01,  1.19it/s]

   现有数据行数: 2160
📖 读取新文件: d:/workspace/xiaoyao/redis/minutely/688275.XSHG_merged.parquet
   新数据行数: 240
🔄 合并数据中...
🧹 去重处理...
📅 按日期排序...
📈 合并后总行数: 2400
💾 保存合并结果: d:/workspace/xiaoyao/data/stock_minutely_price/688275.XSHG.parquet
✅ 合并完成！文件大小: 0.06 MB

📋 验证结果:
   最终行数: 2400
   日期范围: 2025-09-19 00:00:00 到 2025-10-10 00:00:00
   股票数量: 1
已删除：D:\workspace\xiaoyao\redis\minutely\stock_minutely_price_688275.XSHG_20250919.csv
📁 开始处理目录: ./minutely
📊 找到 1 个 CSV 文件
正在处理 (1/1): stock_minutely_price_002119.XSHE_20250919.csv
  ✅ 成功读取 240 条记录

📊 合并所有数据...
📈 总计 240 条记录（去重后）
✅ 成功保存到: ./minutely/002119.XSHE_merged.parquet
📊 文件大小: 0.01 MB
📊 开始合并 parquet 文件...
📖 读取现有文件: d:/workspace/xiaoyao/data/stock_minutely_price/002119.XSHE.parquet
   现有数据行数: 2160
📖 读取新文件: d:/workspace/xiaoyao/redis/minutely/002119.XSHE_merged.parquet
   新数据行数: 240
🔄 合并数据中...
🧹 去重处理...
📅 按日期排序...
📈 合并后总行数: 2400
💾 保存合并结果: d:/workspace/xiaoyao/data/stock_minutely_price/002119.XSHE.parquet
✅ 合并完成！文件大小: 0.04 MB

📋 验证结果:
   最终行数: 2400
   日期范围:

合并进度:   2%|▏         | 78/3937 [00:40<43:14,  1.49it/s]

In [5]:
# from tqdm import tqdm


# if __name__ == "__main__":

#     # 从配置文件读取Redis连接信息
#     with open('redis.conf', 'r') as f:
#         for line in f:
#             if line.startswith('host='):
#                 host = line.split('=')[1].strip()
#             elif line.startswith('port='):
#                 port = int(line.split('=')[1].strip())
#             elif line.startswith('password='):
#                 password = line.split('=')[1].strip()
#     # 初始化Redis发送端
#     sender = RemoteSender(host=host, port=port, password=password)
    
#     # 定义日期范围：从20250516到20250923
#     start_date = '20250922'#(df['date'].max() + timedelta(days=1)).strftime('%Y%m%d')
#     # 获取当日日期-1，是end_date
#     end_date = '20250926'#(datetime.today() - timedelta(days=1)).strftime('%Y%m%d')
    
    
#     daily_df = pd.read_parquet('../data/stock_daily_price.parquet')
#     stock_code_list = daily_df['stock_code'].unique()

#     # 循环调用获取每日数据
#     for stock_code in tqdm(stock_code_list):
#         # 读取已存在的parquet文件
#         start_date2 = start_date

#         existing_file = f"d:/workspace/xiaoyao/redis/minutely/{stock_code}.parquet"
#         # if os.path.exists(existing_file):
#         #     existing_df = pd.read_parquet(existing_file)
#         #     # 读取existing_df的date列的最大date
#         #     max_date = existing_df['date'].max()
#         #     start_date2 = (datetime.strptime(max_date, '%Y%m%d') + timedelta(days=1)).strftime('%Y%m%d')
        
#         date_list = generate_date_range(start_date2, end_date)
#         print(f"=== 共需获取 {len(date_list)} 天的数据 ===")

#         for i, date in enumerate(date_list, 1):
#             try:
#                 # 调用远程函数获取当日数据
#                 csv_data = sender.call_remote_function('fetch_minute_stock_data', date,[stock_code])
#                 # 保存为CSV文件，文件名包含日期
#                 sender.save_to_csv(csv_data, f'./minutely/stock_minutely_price_{stock_code}_{date}.csv')
#                 # 适当延迟，避免请求过于频繁
#                 time.sleep(0.05)
#             except Exception as e:
#                 print(f"❌ {date} 处理失败：{e}")
#                 # 失败后也延迟一下，避免快速重试导致的问题
#                 time.sleep(1)
#         # 下载完成后对csv进行合并
#         success = merge_stock_csv_to_parquet('./minutely/', stock_code, f'./minutely/{stock_code}_merged.parquet')
        
#         if not success:
#             print(f"❌ {stock_code} 合并失败")
        
#         # 将生成的parquet进行合并
#         merge_parquet_files(stock_code)


#         # 删除指定的parquet文件
#         file_to_delete = f'./minutely/{stock_code}_merged.parquet'
#         if os.path.exists(file_to_delete):
#             os.remove(file_to_delete)


#         file_to_delete = f'./minutely/{stock_code}.parquet'
#         if os.path.exists(file_to_delete):
#             os.remove(file_to_delete)

#         # 删除无用的csv
#         delete_stock_csv_files(stock_code)

#     print("\n=== 所有日期处理完成 ===")